# NYC Citi Bike - Data calculation for Map
We downloaded data from 
First we import data from CSV file

In [1]:
from csv import DictReader

the_reader = DictReader(open('data/201803_citibikenyc_tripdata.csv', 'r'))
data = []
for line_dict in the_reader:
    data.append(line_dict)

In [2]:
data[0]

OrderedDict([('tripduration', '455'),
             ('starttime', '2018-03-01 00:00:03'),
             ('stoptime', '2018-03-01 00:07:39'),
             ('start station id', '405'),
             ('start station name', 'Washington St & Gansevoort St'),
             ('start station latitude', '40.739323'),
             ('start station longitude', '-74.008119'),
             ('end station id', '368'),
             ('end station name', 'Carmine St & 6 Ave'),
             ('end station latitude', '40.73038599'),
             ('end station longitude', '-74.00214988'),
             ('bikeid', '17629'),
             ('name_localizedValue0', 'Annual Membership from Citi Bike App'),
             ('usertype', 'Subscriber'),
             ('birth year', '1993'),
             ('gender', '1')])

We selected from the dataset only relevant informations about the stations and sorted stations based on their names in alphabetical order.

In [5]:
##get list of stations
import collections
dic_stations = {}

for item in data:
    dic_stations[item['start station name']] =  {'name': item['start station name'], 'lat':  item['start station latitude'],'lon': item['start station longitude']}
    dic_stations[item['end station name']] = {'name': item['end station name'], 'lat':  item['end station latitude'],'lon': item['end station longitude']}

# list of stations used for visualisation    
simple_stations = sorted(dic_stations.values(), key=lambda x: x['name'] )

# dictionary of station used for futher calculation
dic_stations = collections.OrderedDict(sorted(dic_stations.items(), key=lambda x: x[1]['name'] ))

print(simple_stations[0])


{'name': '1 Ave & E 110 St', 'lat': '40.7923272', 'lon': '-73.9383'}


In [6]:
i=0
#station_sorted = sorted(stations.items(), key=lambda x: x[1]['name'])
stations = dic_stations

#added index
for station in stations:
    stations[station]['o'] = i
    i += 1
print(stations['1 Ave & E 110 St'])

{'name': '1 Ave & E 110 St', 'lat': '40.7923272', 'lon': '-73.9383', 'o': 0}


In [18]:
len(stations)

771

We filter only rents which begin on 1.3.2018

In [7]:
firstday = []
for rent in data:
    if rent['starttime'][:10] == '2018-03-06':
        firstday.append(rent)
        

In [121]:
len(firstday)

42688

In [8]:
#Function which translate time to the index of the bucket. Bucket are 10 minute blocks starting at 00:00

def timeStrToBucket (time_str):
    hours = int(time_str[11:13])
    minutes = int( time_str[14:16])
    index = (hours*6) + int(minutes / 10)
    return index

timeStrToBucket("2018-03-01 00:00:03")

0

I created nested list with dimension 144x771. Every cell contain pair [# rented, # returned] bikes. Rows of the nested list (first dimetion) are equivalent to the 10 minutes time blocks starting at 00:00. Columns (second dimention) are equivalent to order of station.

In [9]:
buckets = [None] * 144

for i in range(144):
    buckets[i] = []
    for j in range(771):
        buckets[i].append([0,0])


for rent in firstday:
    start_station = rent['start station name']
    end_station = rent['end station name']
    
    start_index = timeStrToBucket(rent['starttime'])
    stop_index = timeStrToBucket(rent['stoptime'])
    #print('1st: {} id: {}; 2st: {} id: {}'.format(start_station, start_index, end_station, stop_index))
    #print(start_station, start_index, end_station, stop_index)
    #print(start_station, end_station, start_index, stop_index)
    buckets[start_index][stations[start_station]['o']][0] += 1
    buckets[stop_index][stations[end_station]['o']][1] += 1

#print(buckets)

Each row we sum with previous rows, so we could derermine how many bikes were rented and returned to that station from begining of the day to the specific time block, including it.

In [10]:
#buckets for coloring circles
from copy import deepcopy
sumbuckets = deepcopy(buckets)

for column in range(771):
    for row in range(143):
        sumbuckets[row+1][column][0] += sumbuckets[row][column][0]
        sumbuckets[row+1][column][1] += sumbuckets[row][column][1]
        
print(sumbuckets)

[[[0, 0], [0, 0], [0, 0], [1, 0], [0, 0], [1, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [1, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 1], [1, 1], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 1], [0, 0], [0, 0], [0, 0], [1, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 1], [0, 0], [0, 0], [0, 0], [0, 0], [0, 1], [0, 0], [0, 0], [0, 1], [0, 0], [1, 0], [0, 0], [0, 0], [0, 0], [0, 0]

The maximum number of rented and returned bikes in one station in whole day

In [125]:
max1 = max2 =0
for pair in sumbuckets[143]:
    if pair[0] > max1: max1 = pair[0]
    if pair[1] > max2: max2 = pair[1]
        
print(max1, max2)

579 575


Export data for map visualisation

In [28]:
import json
export = {
    'max_out': max1,
    'max_in': max2,
    'stations': simple_stations,
    "values": sumbuckets,
}
with open("data/stations_rents_outin.json", 'w') as file:
    json.dump(export, file)

We calculated data for the bar chart under the map. Each cell of the list contain how many rents were realised in the 10 minute time block starting at 00:00

In [11]:
number_of_rents = [0] * 144
old_index = 0
for rent in firstday:
    index = timeStrToBucket(rent['starttime'])
    if index != old_index:
        old_index = index
    number_of_rents[old_index] += 1
    
print(number_of_rents)

[49, 31, 28, 32, 35, 25, 19, 13, 11, 17, 13, 11, 6, 9, 8, 10, 8, 1, 10, 8, 7, 6, 2, 5, 1, 2, 6, 8, 13, 17, 14, 25, 53, 67, 87, 94, 114, 162, 213, 213, 291, 286, 327, 385, 410, 509, 589, 599, 696, 782, 870, 913, 960, 765, 755, 611, 545, 512, 470, 415, 339, 269, 298, 265, 273, 269, 250, 260, 240, 289, 316, 290, 258, 317, 332, 336, 337, 292, 375, 311, 327, 346, 336, 333, 384, 376, 357, 338, 343, 346, 347, 378, 350, 337, 436, 364, 388, 377, 428, 473, 463, 560, 661, 767, 762, 731, 722, 700, 728, 785, 714, 678, 641, 550, 539, 482, 449, 416, 373, 351, 332, 301, 329, 267, 266, 218, 245, 223, 231, 188, 189, 165, 178, 152, 130, 127, 110, 115, 93, 80, 76, 71, 59, 48]


In [ ]:
with open("data/rents_timeline.json", 'w') as file:
    json.dump(number_of_rents, file)